In [1]:
import codecs
import pandas as pd
import os
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from os import listdir
from os.path import isfile, join

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [ ]:
#  options1: Vertex: category, cstr. Edge: orientation, complexite
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

transactions = list()
for input_file in input_files:
    transaction = list()
    #if input_file != 'F00040 adressage_Nm.txt':
        #continue
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').replace(' ','').split('\t')
                
                '''orientation = line_elements[21]
                if orientation == 'as2de' or orientation == 'as2des' or orientation == 'de2as' or orientation == 'des2as':
                    orientation = 'direct'
                complexity = line_elements[19]'''
                lex = list()
                lex.append(line_elements[8] + '_' + line_elements[14])
                lex.append(line_elements[10] + '_' + line_elements[17])
                if line_elements[21] == 'as2de' or line_elements[21] == 'as2des':
                    item = lex[0] + ' -> ' + line_elements[19] + ' -> ' + lex[1]
                elif line_elements[21] == 'de2as' or line_elements[21] == 'des2as':
                    item = lex[1] + ' -> ' + line_elements[19] + ' -> ' + lex[0]
                else:
                    lex.sort()
                    item = lex[0] + ' - ' + line_elements[19] + '_' + line_elements[21] + ' - ' + lex[1]
                transaction.append(item)
                lex.clear()
    transactions.append(transaction)
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.index = input_files
df

In [2]:
#  options1: Vertex: semty, cstr. Edges: rel_sem
input_dir = 'families'
input_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]

transactions = list()
for input_file in input_files:
    transaction = list()
    #if input_file != 'F00040 adressage_Nm.txt':
        #continue
    with codecs.open(join(input_dir, input_file), 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f):
            if line_num >= 2:
                line_elements = line.replace('\n','').replace(' ','').split('\t')
                if line_elements[23] == '' or line_elements[25] == '' or line_elements[36] == '':
                    continue
                
                lex1 = list()
                lex2 = list()
                if '|' in line_elements[23]:
                    lex1 = line_elements[23].split('|')
                else:
                    lex1.append(line_elements[23])
                if '|' in line_elements[25]:
                    lex2 = line_elements[25].split('|')
                else:
                    lex2.append(line_elements[25])
                if line_elements[21] == 'as2de' or line_elements[21] == 'as2des':
                    for l1 in lex1:
                        for l2 in lex2:
                            item = l1 + '_' + line_elements[14] + ' -> ' + line_elements[36] + ' -> ' + l2 + '_' + line_elements[17]
                elif line_elements[21] == 'de2as' or line_elements[21] == 'des2as':
                    for l1 in lex1:
                        for l2 in lex2:
                            item = l2 + '_' + line_elements[17] + ' -> ' + line_elements[36] + ' -> ' + l1 + '_' + line_elements[14]
                else:
                    for l1 in lex1:
                        for l2 in lex2:
                            l12 = sorted([l1 + '_' + line_elements[14], l2 + '_' + line_elements[17]])
                            item = l12[0] + ' - ' + line_elements[36] + ' - ' + l12[1]
                transaction.append(item)
                lex1.clear()
                lex1.clear()
    transactions.append(transaction)
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.index = input_files
df

,Abstract_entity_X -> synonymie -> Modifier_Xal,Abstract_entity_X -> synonymie -> Modifier_Xoïdal,Abstract_entity_Xion - synonymie - Modifier_Xoral,Abstract_entity_Xé - synonymie - Modifier_Xal,Act_X -> synonymie -> Modifier_Xal,Act_Xage - synonymie - Modifier_Xal,Act_Xat - synonymie - Modifier_Xal,Act_Xeté - synonymie - Modifier_Xal,Act_Xie - synonymie - Modifier_Xal,Act_Xion - synonymie - Modifier_Xal,...,Situation_Xage - synonymie - Situation_Xment,Situation_Xage - synonymie - Situation_Xonner,Situation_Xage -> synonymie -> Situation_X,Situation_Xerie - synonymie - Situation_Xiser,Situation_Xion - synonymie - Situation_Xion,Situation_Xion - synonymie - Situation_Xment,Situation_Xiser - synonymie - Situation_Xisme,State_X -> synonymie -> Modifier_Xal,Stative-situation_X -> proto-agent -> Artifact_Xeur,Substance_X -> synonymie -> Modifier_Xal
F00000 abaissable_Adj.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F00001 abandon_Nm.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F00002 abat_Nm.txt,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
F00003 abcès_Nm.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F00004 abîme_Nm.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F13174 volubilaire_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F13175 volvaire_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F13176 blessure_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
F13177 Wikiversité_Nf.txt,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [3]:
frequent_itemsets = apriori(df, min_support=0.04, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets['# of families'] = frequent_itemsets['support'].apply(lambda x: x*df.shape[0])
frequent_itemsets

,support,itemsets,length,# of families
0,0.055771,(Artifact_Xeur - proto-agent - Situation_Xage),1,735.0
1,0.045148,(Artifact_Xeur - proto-agent - Situation_Xion),1,595.0
2,0.075651,(Artifact_Xeur - synonymie - Artifact_Xeuse),1,997.0
3,0.052660,(Situation_X - synonymie - Situation_X),1,694.0
4,0.170650,(Situation_X -> proto-agent -> Artifact_Xeur),1,2249.0
5,0.080886,(Situation_X -> proto-agent -> Artifact_Xeuse),1,1066.0
6,0.190303,(Situation_X -> synonymie -> Situation_Xage),1,2508.0
7,0.196070,(Situation_X -> synonymie -> Situation_Xion),1,2584.0
8,0.163366,(Situation_X -> synonymie -> Situation_Xment),1,2153.0
9,0.055543,"(Situation_X -> proto-agent -> Artifact_Xeur, ...",2,732.0


In [4]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.9)
#rules['antecedent length'] = rules['antecedents'].apply(lambda x: len(x))
#rules['consequent length'] = rules['consequents'].apply(lambda x: len(x))
rules.drop(columns=['antecedent support', 'consequent support', 'lift', 'leverage', 'conviction'], inplace=True)
rules

,antecedents,consequents,support,confidence
0,(Artifact_Xeur - proto-agent - Situation_Xage),(Situation_X -> proto-agent -> Artifact_Xeur),0.055543,0.995918
1,(Artifact_Xeur - proto-agent - Situation_Xage),(Situation_X -> synonymie -> Situation_Xage),0.054936,0.985034
2,(Artifact_Xeur - proto-agent - Situation_Xion),(Situation_X -> proto-agent -> Artifact_Xeur),0.040671,0.900840
3,(Artifact_Xeur - proto-agent - Situation_Xion),(Situation_X -> synonymie -> Situation_Xion),0.042264,0.936134
4,(Artifact_Xeur - synonymie - Artifact_Xeuse),(Situation_X -> proto-agent -> Artifact_Xeur),0.074892,0.989970
5,(Artifact_Xeur - synonymie - Artifact_Xeuse),(Situation_X -> proto-agent -> Artifact_Xeuse),0.074512,0.984955
6,(Situation_X -> proto-agent -> Artifact_Xeuse),(Artifact_Xeur - synonymie - Artifact_Xeuse),0.074512,0.921201
7,(Situation_X -> proto-agent -> Artifact_Xeuse),(Situation_X -> proto-agent -> Artifact_Xeur),0.075651,0.935272
8,"(Situation_X -> synonymie -> Situation_Xage, A...",(Situation_X -> proto-agent -> Artifact_Xeur),0.054936,1.000000
9,"(Situation_X -> proto-agent -> Artifact_Xeur, ...",(Situation_X -> synonymie -> Situation_Xage),0.054936,0.989071


In [5]:
families_ant_con = list()
families_ant = list()
for chosen_rule_id in rules.index:
    set_of_antecedents = rules.loc[chosen_rule_id, 'antecedents']
    set_of_consequents = rules.loc[chosen_rule_id, 'consequents']
    conditions_ant = list()
    conditions_ant_con = list()
    for ante in set_of_antecedents:
        conditions_ant.append(df[ante])
        conditions_ant_con.append(df[ante])
    for cons in set_of_consequents:
        conditions_ant.append(~df[cons])
        conditions_ant_con.append(df[cons])

    joined_ant = pd.Series([True] * df.shape[0], index=df.index)
    for c in conditions_ant:
        joined_ant &= c
    indices = df.loc[joined_ant].index
    ret_str = str(len(indices)) + ' families: '
    for ind in indices:
        ret_str += ind.split(' ')[0] + ' '
    families_ant.append(ret_str)

    joined_ant_con = pd.Series([True] * df.shape[0], index=df.index)
    for c in conditions_ant_con:
        joined_ant_con &= c
    indices = df.loc[joined_ant_con].index
    ret_str = str(len(indices)) + ' families: '
    for ind in indices:
        ret_str += ind.split(' ')[0] + ' '
    families_ant_con.append(ret_str)
rules['families with ante & cons'] = families_ant_con
rules['families with ante only'] = families_ant
rules

,antecedents,consequents,support,confidence,families with ante & cons,families with ante only
0,(Artifact_Xeur - proto-agent - Situation_Xage),(Situation_X -> proto-agent -> Artifact_Xeur),0.055543,0.995918,732 families: F00002 F00007 F00020 F00021 F000...,3 families: F05378 F05720 F07317
1,(Artifact_Xeur - proto-agent - Situation_Xage),(Situation_X -> synonymie -> Situation_Xage),0.054936,0.985034,724 families: F00002 F00007 F00020 F00021 F000...,11 families: F00740 F03340 F05107 F05378 F0547...
2,(Artifact_Xeur - proto-agent - Situation_Xion),(Situation_X -> proto-agent -> Artifact_Xeur),0.040671,0.900840,536 families: F00012 F00017 F00033 F00054 F000...,59 families: F01207 F01231 F01480 F01753 F0195...
3,(Artifact_Xeur - proto-agent - Situation_Xion),(Situation_X -> synonymie -> Situation_Xion),0.042264,0.936134,557 families: F00012 F00017 F00033 F00054 F000...,38 families: F01207 F01480 F01824 F01958 F0209...
4,(Artifact_Xeur - synonymie - Artifact_Xeuse),(Situation_X -> proto-agent -> Artifact_Xeur),0.074892,0.989970,987 families: F00001 F00002 F00006 F00007 F000...,10 families: F00427 F01977 F03162 F03293 F0369...
5,(Artifact_Xeur - synonymie - Artifact_Xeuse),(Situation_X -> proto-agent -> Artifact_Xeuse),0.074512,0.984955,982 families: F00001 F00002 F00006 F00007 F000...,15 families: F00139 F00828 F00876 F01030 F0103...
6,(Situation_X -> proto-agent -> Artifact_Xeuse),(Artifact_Xeur - synonymie - Artifact_Xeuse),0.074512,0.921201,982 families: F00001 F00002 F00006 F00007 F000...,84 families: F00035 F00155 F00211 F00264 F0026...
7,(Situation_X -> proto-agent -> Artifact_Xeuse),(Situation_X -> proto-agent -> Artifact_Xeur),0.075651,0.935272,997 families: F00001 F00002 F00006 F00007 F000...,69 families: F00035 F00264 F00269 F00320 F0032...
8,"(Situation_X -> synonymie -> Situation_Xage, A...",(Situation_X -> proto-agent -> Artifact_Xeur),0.054936,1.000000,724 families: F00002 F00007 F00020 F00021 F000...,0 families:
9,"(Situation_X -> proto-agent -> Artifact_Xeur, ...",(Situation_X -> synonymie -> Situation_Xage),0.054936,0.989071,724 families: F00002 F00007 F00020 F00021 F000...,8 families: F00740 F03340 F05107 F05479 F05775...


In [6]:
#rules.sort_values(by=['confidence'], ascending=False, inplace=True)
rules.to_csv('rules_sem.csv', sep='\t')